In [2]:
import os
from random import randrange
from glob import glob

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import CheXpertDataset
from model import MultiLabelClassification
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.models import DenseNet121

In [3]:
# Get Data
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '..', 'data')
CheXpert_train_hidden_features_all = np.load(os.path.join(DATA_DIR,'CheXpert_train_hidden_features_all.npy'))
CheXpert_train_labels_all = np.load(os.path.join(DATA_DIR,'CheXpert_train_labels_all.npy'))
CheXpert_valid_hidden_features_all = np.load(os.path.join(DATA_DIR,'CheXpert_valid_hidden_features_all.npy'))
CheXpert_valid_labels_all = np.load(os.path.join(DATA_DIR,'CheXpert_valid_labels_all.npy'))
extra_valid_age_sex_df = pd.read_csv(os.path.join(DATA_DIR,'extra_valid_age_sex.csv'))
extra_valid_hidden_features = np.load(os.path.join(DATA_DIR,'extra_valid_hidden_features.npy'))
extra_valid_labels = np.load(os.path.join(DATA_DIR,'extra_valid_labels.npy'))
extra_valid_images = glob(os.path.join(DATA_DIR, 'extraValid', '*'))

In [3]:
# Add age and sex to hidden feature vectors
extra_valid_age_sex_df = extra_valid_age_sex_df.replace(['F', 'M', 'O'], [0, 1, 2])
extra_valid_age_sex_df = extra_valid_age_sex_df.apply(pd.to_numeric)
extra_valid_age_sex_np = extra_valid_age_sex_df.to_numpy()
hidden_with_age_sex = np.concatenate((extra_valid_hidden_features, extra_valid_age_sex_np), axis=1)
#hidden_with_age_sex = hidden_with_age_sex.astype('float32')
hidden_with_age_sex = extra_valid_hidden_features.astype('float32')

In [4]:
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed = 144
set_all_seeds(seed)

In [38]:
model = DenseNet121(pretrained=True, last_activation=None, activations='relu', num_classes=5)
layer = model._modules.get('classifier')

In [39]:
layer.weight.data

tensor([[ 0.0227,  0.0227,  0.0083,  ...,  0.0207,  0.0236,  0.0252],
        [-0.0015, -0.0172,  0.0225,  ...,  0.0098, -0.0172, -0.0238],
        [ 0.0143, -0.0168, -0.0166,  ..., -0.0078,  0.0134,  0.0264],
        [-0.0165, -0.0143, -0.0177,  ..., -0.0156, -0.0127, -0.0299],
        [ 0.0068,  0.0138, -0.0149,  ..., -0.0242,  0.0066, -0.0110]])

In [40]:
print(layer)

Linear(in_features=1024, out_features=5, bias=True)
